# Introduction and Background

In this notebook, we demonstrate how to adapt the kernel methods shown in the [previous notebook](3_KernelMethods.ipynb) to use sparse kernels.

As for the previous notebooks, for each model, we first go step-by-step through the derivation, with equations, embedded links, and citations supplied where useful. At the end of the notebook, we employ a "Utility Class" for the model, which is found in the utilities folder and contains all necessary functions.

In [ ]:
#!/usr/bin/env python3
import sys

# Maths things
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Local Utilities for Notebook
sys.path.append('../')
from utilities.general import FPS, load_variables, sorted_eig, get_stats
from utilities.plotting import (
    plot_projection, plot_regression, check_mirrors, get_cmaps, table_from_dict
)
from utilities.kernels import linear_kernel, gaussian_kernel, center_kernel
from utilities.classes import KPCA, KRR, SparseKPCA, SparseKRR
from utilities.kpcovr import KPCovR, SparseKPCovR

cmaps = get_cmaps()
plt.style.use('../utilities/kernel_pcovr.mplstyle')
dbl_fig=(2*plt.rcParams['figure.figsize'][0], plt.rcParams['figure.figsize'][1])

First, we must load the data. For a step-by-step explanation of this, please see [Importing Data](X_ImportingData.ipynb).

In [ ]:
var_dict = load_variables()
locals().update(var_dict)

# Constructing a Sparse Kernel

In [ ]:
## Change this cell to change the kernel function throughout

kernel_func = gaussian_kernel
kernel_type = 'gaussian'

## The Nystr&ouml;m Approximation

In sparse kernel methods, an approximate kernel is used in place of the full kernel. This approximate kernel is typically constructed according to the [Nystr&ouml;m approximation](https://en.wikipedia.org/wiki/Low-rank_matrix_approximations#Nystr%C3%B6m_approximation) [(Williams 2001)](http://papers.nips.cc/paper/1866-using-the-nystrom-method-to-speed-up-kernel-machines.pdf),

\begin{equation}
    \mathbf{K} \approx \mathbf{\tilde{K}}_{NN} = \mathbf{K}_{NM} \mathbf{K}_{MM}^{-1} \mathbf{K}_{NM}^T,
\end{equation}

Here, $M$ represents a subset of the $N$ total rows/columns of the kernel matrix, i.e. the kernel between a small **active set** than is selected with subsampling method, like farthest point sampling (FPS) [(Eldar 1997)](https://doi.org/10.1109/83.623193), or a CUR decomposition [(Imbalzano2018)](https://doi.org/10.1063/1.5024611), that is discussed in the [next notebook](5_CUR.ipynb).

In [ ]:
n_active = 20

fps_idxs, _ = FPS(X_train, n_active)
Xsparse = X_train[fps_idxs, :]

In other words, $\mathbf{K}_{NM}$ is the kernel matrix between input data $\mathbf{X}$ and $\mathbf{X_{sparse}}$, a version of $\mathbf{X}$ containing only the active set. $\mathbf{K}_{MM}$ is the matrix containing the kernel evaluated between the active set samples.

We center our kernels with $\mathbf{K}_{MM}$ as our reference, i.e. for any kernel $\mathbf{K}_{NM}$, the centered kernel is:

\begin{equation}
\tilde{\mathbf{K}}_{NM} = 
\mathbf{K}_{NM} -  \mathbf{1}_{NM} \mathbf{K}_{MM} -  \mathbf{K}_{NM}\mathbf{1}_{MM}
+ \mathbf{1}_{NM} \mathbf{K}_{MM} \mathbf{1}_{MM}.
\end{equation}

analogous to what discussed in [the previous notebook](3_KernelMethods.ipynb).

In [ ]:
Kmm_raw = kernel_func(Xsparse, Xsparse)
Kmm = center_kernel(Kmm_raw) 

Knm_train = kernel_func(X_train, Xsparse)
Knm_train = center_kernel(Knm_train, reference=Kmm_raw)

Knm_test = kernel_func(X_test, Xsparse)
Knm_test = center_kernel(Knm_test, reference=Kmm_raw)

## The Explicit RKHS

Sometimes, it might be more convenient to explicitly write out the projection of the training points
on the [RKHS](https://en.wikipedia.org/wiki/Reproducing_kernel_Hilbert_space) defined by the active set.
This is essentially a KPCA built for the active set, that is not truncated to a few eigenvectors,


\begin{equation}
    \mathbf{\Phi}_{NM} = \mathbf{K}_{NM} \mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2}.
\end{equation}

Using this definition it is easy to derive the Nyström approximation: 

\begin{equation}
\mathbf{\tilde{K}}_{NN} = \mathbf{\Phi}_{NM} \mathbf{\Phi}_{NM}^T = 
\mathbf{K}_{NM}  \mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1}  \mathbf{U}_{MM}^T \mathbf{K}_{NM}^T
= \mathbf{K}_{NM} \mathbf{K}_{MM}^{-1} \mathbf{K}_{NM}^T.
\end{equation}

It might be wise to discard some of the smaller eigenvalues. For instance, if it has been centered, $\mathbf{K}_{MM}$ has one _exactly_ zero eigenvalue, and we should take it out of the projection. [(Honeine 2014)](https://arxiv.org/pdf/1407.2904.pdf)

In [ ]:
vmm, Umm = sorted_eig(Kmm, thresh=1e-12)

Phi = np.matmul(Knm_train, Umm[:,:n_active-1])
Phi = np.matmul(Phi, np.diagflat(1.0/np.sqrt(vmm[0:n_active-1])))

## Centering the RKHS

This $\mathbf{\Phi}_{NM}$ is also centered relative to the active set, $\mathbf{X}$ might well be centered otherwise. We must re-center according to the distribution of $\mathbf{X}$.  
We introduce the centered version, $\mathbf{\tilde{\Phi}}_{NM} = \mathbf{\Phi}_{NM} - \mathbf{\bar{\Phi}}_{M}$. If you represent each element of $\mathbf{\Phi}$ in its summation form

\begin{equation}
    \mathbf{\Phi}_{nm} = \frac{1}{\sqrt{\Lambda_{mm}}}\sum_{m'}^M \left(K_{nm'} U_{m'm}\right), 
\end{equation}

then the column means are given by

\begin{equation}
    \mathbf{\bar{\Phi}}_{m} = \frac{1}{\sqrt{\Lambda_{mm}}}\sum_{m'}^M \left(\left(\frac{1}{N}\sum_n^N K_{nm'}\right)U_{m'm} \right), 
\end{equation}

so the centered feature matrix is computed by $\mathbf{K}_{NM}$, centered by the column means of the kernel, as denoted by $\mathbf{\bar{K}}_M$.

\begin{equation}
    \mathbf{\tilde{\Phi}}_{NM} =  \left(\mathbf{K}_{NM} -\mathbf{\bar{K}}_M\right) \mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2}.
\end{equation}

It is best to keep the column mean $\mathbf{\bar{K}}_M$ separate, because it has to be used also when performing out-of-sample embedding, where $\mathbf{K}_{NM}$ would corresponds to the test set kernel. For consistency, $\mathbf{\bar{K}}_M$ must always be the kernel mean associated with the train set.

Alternatively, one can store $\mathbf{\bar{\Phi}}_{M}$ and use it for centering.

**Note**: in the following we often use $\mathbf{\Phi}_{NM}$ and $\mathbf{\tilde{\Phi}}_{NM}$ without the subscripts to indicate the train set features approximated in the active RKHS. 

<!---
\begin{equation}
    \mathbf{\tilde{\Phi}}_{nm} = \frac{1}{\sqrt{\Lambda_{mm}}}\sum_{m'}^M \left(\left(K_{nm'} - \frac{1}{N}\sum_{n'}^N K_{n'm'}\right)U_{m'm}\right)
\end{equation}
\end{comment}
--->

In [ ]:
barKm = np.mean(Kmm, axis=0)

barPhi = np.mean(Phi, axis=0)
Phi -= barPhi

# Sparse KPCA

Sparse kernel principal component analysis (sKPCA) is formulated in the same way as standard KPCA, with the exception that an approximate kernel matrix is used. 

$\mathbf{\tilde{\Phi}}$ is the feature matrix for the train points in the RKHS defined by the $M$ active set. Sparse KPCA can be understood (and derived) as PCA in the active set RKHS, by computing and diagonalising the covariance matrix built from $\mathbf{\Phi}_{NM}$. The covariance should be computed using *centered* kernel features, as discussed above

\begin{equation}
\mathbf{C} = \mathbf{\tilde{\Phi}}^T \mathbf{\tilde{\Phi}} = \mathbf{U}_C \mathbf{\Lambda}_C \mathbf{U}_C^T.
\end{equation}

Note that - as usual - one could also compute the RKHS covariance without explicitly diagonalising $\mathbf{K}_{MM}$, as 

\begin{equation}
\mathbf{C} = 
 \left(\mathbf{K}_{NM} -\mathbf{\bar{K}}_M\right) \mathbf{K}_{MM}^{-1}  \left(\mathbf{K}_{NM} -\mathbf{\bar{K}}_M\right)^T,
\end{equation}

where $\mathbf{\bar{K}}_M$ indicates the centering vector as discussed above.

In [ ]:
C = np.dot(Phi.T, Phi)

v_C, U_C = sorted_eig(C, thresh=0)

## Projecting the Sparse KPCA
Projecting into latent space, we get

\begin{align}
    \mathbf{T} &= \mathbf{\tilde{\Phi}} \hat{\mathbf{U}}_C \\
    &= \left(\mathbf{K}_{NM}- \bar{\mathbf{K}}_M\right)\mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2} \hat{\mathbf{U}}_C \\
    &= \mathbf{K}_{NM}\mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2} \hat{\mathbf{U}}_C -\bar{\mathbf{\Phi}}\hat{\mathbf{U}}_C\\
    &= \mathbf{K}_{NM} \mathbf{P}_{KT} - \mathbf{\bar{T}}
\end{align}

where our sKPCA projector from kernel space $\mathbf{P}_{KT} = \mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2}\mathbf{\hat{U}}_C$, where $\mathbf{\hat{U}}_C$ contains the first $n_{PCA}$ eigenvectors of $\mathbf{C}$. $\mathbf{\bar{T}} = \bar{\mathbf{\Phi}}\hat{\mathbf{U}}_C$ centers in the latent space, and is computed and stored for future use.

In [ ]:
PKT = np.matmul(Umm[:,:n_active-1],\
                    np.diagflat(1.0/np.sqrt(vmm[0:n_active-1])))
PKT = np.matmul(PKT, U_C[:, :n_PCA])
barT = np.matmul(barPhi, U_C[:, :n_PCA])

T_train = np.matmul(Knm_train, PKT) - barT
T_test = np.matmul(Knm_test, PKT) - barT

In [ ]:
fig, axes = plt.subplots(1,2, figsize=dbl_fig)

ref_kpca = KPCA(n_KPCA=n_PCA, kernel_type=kernel_type)
ref_kpca.fit(X_train, K=K_train)
xref = ref_kpca.transform(X_test, K=K_test)

plot_projection(Y_test, check_mirrors(T_test, xref), fig=fig, ax=axes[0], \
                 title="Sparse KPCA on {} Environments".format(Kmm.shape[0]),
                  **cmaps
         )
plot_projection(Y_test,  xref, fig=fig, ax=axes[1], \
                title="KPCA on {} Environments".format(X_train.shape[0]),
               **cmaps
         )

fig.subplots_adjust(wspace=0.4)

We can also reconstruct $\mathbf{X}$ using $\mathbf{P}_{TX} = \mathbf{\Lambda}^{-1}\mathbf{T}^T\mathbf{X}$, as in KPCA.

In [ ]:
PTX = np.matmul(np.diagflat(1.0/(v_C[:n_PCA])),np.matmul(T_train.T, X_train))

Xr_test = np.matmul(T_test, PTX)

## Error and Loss

The same loss functions are used as in KPCA, so we can compare the loss with that of KPCA

In [ ]:
K_approx_train = np.matmul(T_train,T_train.T)

K_test_test = kernel_func(X_test, X_test)
K_test_test = center_kernel(K_test_test)
K_approx_test = np.matmul(T_test,T_test.T)

table_from_dict([ref_kpca.statistics(X_test, Y_test, K=K_test),
                 get_stats(x=X_test, 
                           xr=Xr_test,
                           y=Y_test, 
                           t=T_test, 
                           k=K_test_test, 
                           kapprox=K_approx_test)], 
                 headers = ["KPCA", "sKPCA"], 
                 title="sKPCA")

# Sparse KRR

## Sparse KRR Weights
Let's see how sparsity works out in the case of regression. 

If we now build a (regularized) linear regression in the RKHS we get the loss

\begin{equation}
\ell = \lVert \mathbf{Y} - \mathbf{\Phi}\mathbf{P}_{\Phi Y} \rVert^2 + 
\lambda \lVert\mathbf{P}_{\Phi Y} \rVert^2
\end{equation}

This is solved by 

\begin{equation}
\mathbf{P}_{\Phi Y} = \left(\mathbf{\Phi}^T \mathbf{\Phi}+ \lambda \mathbf{I}\right)^{-1} \mathbf{\Phi}^T \mathbf{Y}
\end{equation}

or, by writing the last $\mathbf{\Phi}^T$ in terms of the kernel:

\begin{equation}
\mathbf{P}_{\Phi Y} = \left(\mathbf{\Phi}^T \mathbf{\Phi}+ \lambda \mathbf{I}\right)^{-1} \mathbf{\Lambda}_{MM}^{-1/2} \mathbf{U}_{MM}^T \mathbf{K}_{NM}^T  \mathbf{Y}
\end{equation}

**Note**: Since (kernel) ridge regression is often performed without centering the kernel, we use the uncentered feature matrix $\mathbf{\Phi}$ instead of $\mathbf{\tilde{\Phi}}$.

In [ ]:
regularization = 1e-12

Let's start from after we've computed our sparse kernels and recompute $\mathbf{\Phi}$.

In [ ]:
%%time

vmm, Umm = sorted_eig(Kmm, thresh=0)

Phi_raw = np.matmul(Knm_train, Umm[:,:n_active-1])
Phi_raw = np.matmul(Phi_raw, np.diagflat(1.0/np.sqrt(vmm[0:n_active-1])))

barKM = np.mean(Knm_train, axis=0)
Phi = Knm_train
Phi = np.matmul(Knm_train, Umm[:,:n_active-1])
Phi = np.matmul(Phi, np.diagflat(1.0/np.sqrt(vmm[0:n_active-1])))
barPhi = np.mean(Phi, axis=0)
Phi -= barPhi

PPY = np.matmul(Phi.T, Phi)
PPY = PPY + regularization*np.eye(Phi.shape[1])
PPY = np.linalg.pinv(PPY)
PPY = np.matmul(PPY, Phi.T)
PPY = np.matmul(PPY, Y_train)

## An Often Cheaper, More Elegant Route

We cast this expression into the more commonly used form by a series of simple manipulations that remove the need for diagonalizing $K_{MM}$ and computing $\mathbf{\Phi}$. First, we redefine the weights so that 

\begin{equation}
\mathbf{\Phi}\mathbf{P}_{\Phi Y} = 
\mathbf{K}_{NM} \mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2} \mathbf{P}_{\Phi Y} = 
\mathbf{K}_{NM} \tilde{\mathbf{P}_{K Y}}.
\end{equation}

Then,

\begin{align}
\tilde{\mathbf{P}_{K Y}}  &= 
\mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2}
\mathbf{P}_{\Phi Y} \\
& = \mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{-1/2}
\left(\mathbf{\Phi}^T \mathbf{\Phi}+ \lambda \mathbf{I}_M\right)^{-1} 
\mathbf{\Lambda}_{MM}^{-1/2}  \mathbf{U}_{MM}^T 
\mathbf{K}_{NM}^T \mathbf{Y}\\
& = 
\left(\mathbf{U}_{MM}\mathbf{\Lambda}_{MM}^{1/2}\mathbf{\Phi}^T \mathbf{\Phi}\mathbf{\Lambda}_{MM}^{1/2}\mathbf{U}_{MM}^T+ \lambda \mathbf{U}_{MM}\mathbf{\Lambda}_{MM}\mathbf{U}_{MM}^T\right)^{-1} 
\mathbf{K}_{NM}^T \mathbf{Y}.
\end{align}

Now, by noting that 

\begin{equation}
\mathbf{U}_{MM} \mathbf{\Lambda}_{MM}^{1/2} 
\mathbf{\Phi}^T \mathbf{\Phi}
\mathbf{\Lambda}_{MM}^{1/2}  \mathbf{U}_{MM}^T  = 
\mathbf{K}_{NM}^T \mathbf{K}_{NM},
\end{equation}

we see that the sparse KRR model weights is computed by

\begin{equation}
\tilde{\mathbf{P}_{K Y}}  = 
\left(\mathbf{K}_{NM}^T \mathbf{K}_{NM}+ \lambda \mathbf{K}_{MM}\right)^{-1} 
\mathbf{K}_{NM}^T \mathbf{Y}.
\end{equation}


In [ ]:
%%time

PKY = np.matmul(Knm_train.T, Knm_train)
PKY = PKY + regularization*Kmm
PKY = np.linalg.pinv(PKY)
PKY = np.matmul(PKY, Knm_train.T)
PKY = np.matmul(PKY, Y_train)

As you see, this trick provides a (in some cases considerable) speed-up.

In [ ]:
Y_skrr_train = np.matmul(Knm_train, PKY)
Y_skrr_test = np.matmul(Knm_test, PKY)

We compare our results with those from KRR.

In [ ]:
fig, axes = plt.subplots(1,2, figsize=dbl_fig)

ref_krr = KRR(regularization=regularization, kernel_type=kernel_type)
ref_krr.fit(X=X_train, Y=Y_train, K=K_train)
Y_krr = ref_krr.transform(X=X_test, K=K_test)

plot_regression(Y_test[:,0], Y_krr[:,0], title="KRR", fig=fig, ax=axes[0], **cmaps)
plot_regression(Y_test[:,0], Y_skrr_test[:,0], title="Sparse KRR on {} Environments".format(n_active), fig=fig, ax=axes[1], **cmaps)


## Error and Loss

Here our loss function is:

\begin{equation}
\ell = \left\lVert \mathbf{Y} - \mathbf{K}_{NM}\mathbf{P}_{KY}\right\rVert^2
\end{equation}

which we compare with KRR.

In [ ]:
table_from_dict([ref_krr.statistics(X_test, Y_test, K=K_test),
                 get_stats(x=X_test, 
                           y=Y_test, 
                           yp = Y_skrr_test,
                          )], 
                 headers = ["KRR", "sKRR"], 
                 title="Ridge Regression")

# Sparse KPCovR

For Sparse KPCovR, instead of using the Nystr&ouml;m approximation as in previous sparse methods, we formulate sparse KPCovR from KPCovR in a similar manner to how we derived feature space PCovR from structure space PCovR in the [PCovR Notebook](2_PrincipalCovariatesRegression.ipynb).

## A (Very) Quick Recap of Sample and Feature Space PCovR
In PCovR, we maximize the similarity

\begin{equation}
\rho = \operatorname{Tr}\left(\mathbf{V}^T\mathbf{\tilde{K}}\mathbf{V}\right),
\end{equation}

by taking as our whitened projection $\mathbf{V} = \mathbf{XP}_{XV}$ the eigenvectors corresponding to the $n_{PCA}$ largest eigenvalues of

\begin{equation}
    \mathbf{\tilde{K}} = \alpha {\mathbf{X} \mathbf{X}^T}
    + (1 - \alpha) {\hat{\mathbf{Y}} \hat{\mathbf{Y}}^T},
\end{equation}

which combines correlations between the samples in feature and property space. 

If the number of features is less than the number of samples, we can equivalently rewrite our similarity function as

\begin{align}
\rho &= \operatorname{Tr}\left(\mathbf{P}_{XV}^T\mathbf{C}^{1/2}\mathbf{C}^{-1/2}\mathbf{X}^T\mathbf{\tilde{K}}\mathbf{X}\mathbf{C}^{1/2}\mathbf{C}^{-1/2}\mathbf{P}_{XV}\right)
\end{align}

and diagonalize a modified covariance

\begin{equation}
\tilde{\mathbf{C}} = \mathbf{C}^{-1/2}\mathbf{X}^T\mathbf{\tilde{K}}\mathbf{X}\mathbf{C}^{-1/2}
\end{equation}

where $\mathbf{C} = \mathbf{X}^T\mathbf{X}$ to avoid diagonalizing the $n_{samples} \times n_{samples}$ matrix $\tilde{\mathbf{K}}$.

## Now we just do feature-space PCovR in the RKHS

In KPCovR, we maximize the similarity

\begin{equation}
\rho = \operatorname{Tr}\left(\mathbf{V}^T\mathbf{\tilde{K}}\mathbf{V}\right),
\end{equation}

however here $\mathbf{V} = \mathbf{KP}_{KT}$.
We compute the projection in feature space by maximizing:

\begin{align}
\rho = \operatorname{Tr}\left(\mathbf{P}_{KT}^T\mathbf{\tilde{\Phi}}\mathbf{\tilde{\Phi}}^T\mathbf{\tilde{K}}\mathbf{\tilde{\Phi}}\mathbf{\tilde{\Phi}}^T\mathbf{P}_{KT}\right)
\end{align}

where $\mathbf{K} = \mathbf{\tilde{\Phi}}\mathbf{\tilde{\Phi}}^T$.
It would make sense to use $\mathbf{\tilde{\Phi}}^T\mathbf{\tilde{K}}\mathbf{\tilde{\Phi}}$ as our sparse "kernel", but we must insert an identity to ensure that its eigenvectors are orthogonal. We use the covariance, defined as $\mathbf{C} = \mathbf{\tilde{\Phi}}^T\mathbf{\tilde{\Phi}}$, giving:

\begin{align}
\rho=\operatorname{Tr}\left(\mathbf{P}_{KT}^T\mathbf{\tilde{\Phi}}\mathbf{C}^{1/2}
\mathbf{C}^{-1/2}
\mathbf{\tilde{\Phi}}^T\mathbf{\tilde{K}}\mathbf{\tilde{\Phi}}\mathbf{C}^{-1/2}
\mathbf{C}^{1/2}
\mathbf{\tilde{\Phi}}^T\mathbf{P}_{KT}\right)
\end{align}

which ensures orthogonality, as 

\begin{align}
\left(\mathbf{P}_{KT}^T\mathbf{\tilde{\Phi}}\mathbf{C}^{1/2}
\mathbf{C}^{1/2}
\mathbf{\tilde{\Phi}}^T\mathbf{P}\right)
&=\left(\mathbf{P}^T\mathbf{\tilde{\Phi}}\mathbf{\tilde{\Phi}}^T\mathbf{\tilde{\Phi}}\mathbf{\tilde{\Phi}}^T\mathbf{P}\right)\\
&=\left(\mathbf{P}^T\mathbf{K}^T\mathbf{K}\mathbf{P}\right)\\
&=\left(\mathbf{V}^T\mathbf{V}\right)\\
&=\mathbf{I}
\end{align}

In [ ]:
C = np.dot(Phi.T, Phi)

v_C, U_C = sorted_eig(C, thresh=0)
U_C = U_C[:, v_C>0]
v_C = v_C[v_C>0]


In analogy with to feature-space PCovR, we define

\begin{equation}
\mathbf{\tilde{C}} = \mathbf{C}^{-1/2}
\mathbf{\tilde{\Phi}}^T\mathbf{\tilde{K}}\mathbf{\tilde{\Phi}}\mathbf{C}^{-1/2}
\end{equation}

which evaluates to
\begin{equation}
\mathbf{\tilde{C}} = \alpha \frac{\mathbf{C}} {\operatorname{Tr}(\mathbf{C})/N} + (1 - \alpha) \mathbf{C}^{-1/2}\mathbf{\tilde{\Phi}}^T\mathbf{\hat{Y}}
\mathbf{\hat{Y}}^T\mathbf{\tilde{\Phi}}\mathbf{C}^{-1/2},
\end{equation}

**Note**: for consistency, we cannot substitute $\mathbf{\hat{Y}}$ with $\mathbf{Y}_{sKRR} = \mathbf{K}_{NM}\left(\mathbf{K}_{NM}^T\mathbf{K}_{NM}+ \lambda \mathbf{K}_{MM}\right)\mathbf{K}_{NM}^T \mathbf{Y}$ unless we use a centered feature matrix for sparse KRR. Given that we need to compute the eigenvalue decomposition of $\mathbf{K}_{MM}$ to orthogonalize the modified covariance, we can instead use the linear regression solution computed directly in active points RKHS: $\mathbf{Y}_{sKRR} = \mathbf{\tilde{\Phi}}\left(\mathbf{C}+ \lambda \mathbf{I}\right)^{-1}\mathbf{\tilde{\Phi}}^T \mathbf{Y}$.

<!---
\begin{equation}
\mathbf{\tilde{C}} = \alpha \frac{\mathbf{C}} {\operatorname{Tr}(\mathbf{C})/N} + (1 - \alpha) \mathbf{C}^{-1/2}
\mathbf{\Lambda}_{MM}^{-1/2}
\mathbf{U}_{MM}^T
\mathbf{K}_{NM}^T\mathbf{K}_{NM}
\left(\mathbf{K}_{NM}^T\mathbf{K}_{NM}+ \lambda \mathbf{K}_{MM}\right)^{-1}
\mathbf{K}_{NM}^T 
\mathbf{Y}
\mathbf{Y}^T
\mathbf{K}_{NM}
\left(\mathbf{K}_{NM}^T\mathbf{K}_{NM}+ \lambda \mathbf{K}_{MM}\right)^{-1}
\mathbf{K}_{NM}^T\mathbf{K}_{NM}
\mathbf{U}_{MM}
\mathbf{\Lambda}_{MM}^{-1/2}
\mathbf{C}^{-1/2}
\end{equation}

\begin{equation}
\mathbf{\tilde{C}} = \alpha \frac{\mathbf{C}} {\operatorname{Tr}(\mathbf{C})/N} + (1 - \alpha) \mathbf{C}^{-1/2}
\mathbf{\Lambda}_{MM}^{-1/2}
\mathbf{U}_{MM}^T
\left(\mathbf{I}_{M}+ \lambda 
\mathbf{K}_{MM}\left(\mathbf{K}_{NM}^T\mathbf{K}_{NM}\right)^{-1}
\right)^{-1}
\mathbf{K}_{NM}^T 
\mathbf{Y}
\mathbf{Y}^T
\mathbf{K}_{NM}
\left(\mathbf{I}_{M}+ \lambda 
\mathbf{K}_{MM}\left(\mathbf{K}_{NM}^T\mathbf{K}_{NM}\right)^{-1}
\right)^{-1}
\mathbf{U}_{MM}
\mathbf{\Lambda}_{MM}^{-1/2}
\mathbf{C}^{-1/2}
\end{equation}
--->

In [ ]:
alpha = 0.5
regularization=1e-6

Csqrt = np.matmul(np.matmul(U_C, np.diagflat(np.sqrt(v_C))), U_C.T)
iCsqrt = np.matmul(np.matmul(U_C, np.diagflat(1.0/np.sqrt(v_C))), U_C.T)

C_pca = C / (np.trace(C)/C.shape[0])

C_lr = np.linalg.pinv(C + regularization*np.eye(C.shape[0]))
C_lr = np.matmul(Phi, C_lr)
C_lr = np.matmul(Phi.T, C_lr)
C_lr = np.matmul(iCsqrt, C_lr)
C_lr = np.matmul(C_lr, Phi.T)
C_lr = np.matmul(C_lr, Y_train.reshape(-1,Y_train.shape[-1]))
C_lr = np.matmul(C_lr, C_lr.T)

Ct = alpha*C_pca + (1-alpha)*C_lr

We can then find the eigendecomposition of 
$\mathbf{\tilde{C}}=
\mathbf{U}_\mathbf{\tilde{C}}\mathbf{\Lambda}_\mathbf{\tilde{C}}\mathbf{U}_\mathbf{\tilde{C}}^T$  and 
solve for $\mathbf{P}_{\tilde{\Phi} T}$ (again analogous to feature-space PCovR, swapping $\mathbf{\Phi}$ for $\mathbf{X}$): 

\begin{equation}
\mathbf{P}_{\tilde{\Phi} T}=\mathbf{C}^{-1/2}\mathbf{\hat{U}}_\mathbf{\tilde{C}}\mathbf{\hat{\Lambda}}_\mathbf{\tilde{C}}^{1/2} 
\end{equation}

where the $\hat{\cdot}$ decoration denotes a truncation to $n_{PCA}$ components, as usual.

In [ ]:
v_Ct, U_Ct = sorted_eig(Ct, thresh=0)

PPT = np.matmul(iCsqrt, U_Ct[:, :n_PCA])
PPT = np.matmul(PPT, np.diag(np.sqrt(v_Ct[:n_PCA])))
v_Ct.shape

## Projecting into Latent Space

Our projection in feature space takes the form:

\begin{equation}
\mathbf{T} = \mathbf{\tilde{\Phi}}_{NM}\mathbf{P}_{\tilde{\Phi} T}
\end{equation}

If we want to project using a kernel rather than a feature space vector, this becomes:

\begin{align}
\mathbf{T} &=  \left(\mathbf{K}_{NM}-\mathbf{\bar{K}}_M\right)\mathbf{U}_{MM}\mathbf{\Lambda}_{MM}^{-1/2}\mathbf{P}_{\tilde{\Phi} T} \\
&=\mathbf{K}_{NM}\mathbf{U}_{MM}\mathbf{\Lambda}_{MM}^{-1/2}\mathbf{C}^{-1/2}\mathbf{\hat{U}}_\mathbf{\tilde{C}}\mathbf{\hat{\Lambda}}_\mathbf{\tilde{C}}^{1/2} - \mathbf{\bar{\Phi}}\mathbf{P}_{\tilde{\Phi} T} \\
&= \mathbf{K}_{NM}\mathbf{P}_{KT} -  \mathbf{\bar{T}}
\end{align}

where $\mathbf{P}_{K T} = \mathbf{P}_{K\Phi} \mathbf{P}_{\Phi T} = 
\mathbf{U}_{MM}\mathbf{\Lambda}_{MM}^{-1/2}\mathbf{C}^{-1/2}\mathbf{\hat{U}}_\mathbf{\tilde{C}}\mathbf{\hat{\Lambda}}_\mathbf{\tilde{C}}^{1/2} $.

In [ ]:
PKT = np.matmul(Umm[:, :n_active-1], np.diagflat(1/np.sqrt(vmm[:n_active-1])))
PKT = np.matmul(PKT, PPT)
barT = np.matmul(barPhi, PPT)

In [ ]:
T =  np.matmul(Knm_train, PKT) - barT

In [ ]:
T_skpcovr_test = np.matmul(Knm_test, PKT) - barT

We again compare to the non-sparse kernel version, giving

In [ ]:
fig, axes = plt.subplots(1,2, figsize=dbl_fig)

ref = KPCovR(alpha=alpha, n_PCA=2, kernel_type=kernel_type)
ref.fit(X_train, Y_train)
xref, yref, r = ref.transform(X_test)

plot_projection(Y_test, check_mirrors(T_skpcovr_test, xref), fig=fig, ax=axes[0], title = "Sparse KPCovR", **cmaps)
plot_projection(Y_test, xref, fig=fig, ax=axes[1], title = "KPCovR", **cmaps)

plt.show()

## Predicting the Properties

Property prediction takes the exact same form as in KPCovR, except with $\mathbf{T}$ supplied by our sparse construction.

In [ ]:
PTY = np.matmul(T.T, T)
PTY = np.linalg.pinv(PTY)
PTY = np.matmul(PTY, T.T)
PTY = np.matmul(PTY, Y_train)

In [ ]:
Ypred = np.matmul(Knm_test, PKT)
Ypred = np.matmul(Ypred, PTY)

fig, axes = plt.subplots(1,2, figsize=dbl_fig)

plot_regression(Y_test[:,0], Ypred[:,0], fig=fig, ax=axes[0], title = f"Sparse KPCovR with {n_active} Environments", **cmaps)
plot_regression(Y_test[:,0], yref[:,0], fig=fig, ax=axes[1], title = "KPCovR", **cmaps)

plt.show()

# Next: CUR Decomposition and Feature Selection

Continue on to the [next notebook](5_CUR.ipynb).

# The Utility Classes

Classes from the utility module enable computing SparseKPCA, SparseKRR, and SparseKPCovR with a scikit.learn-like syntax. `SparseKPCovR` is located in `utilities/kpcovr.py`.

In [ ]:
from utilities.classes import SparseKPCA, SparseKRR
from utilities.kpcovr import SparseKPCovR

**Important Note**: In all sparse kernel classes, the functions `fit`, `transform`, and `statistics` either computes the designated kernels for the supplied $\mathbf{X}$ data or uses the provided precomputed kernels. The kernel are **always** a keyword argument (e.g. `model.fit(Kmm=Kmm, Knm=Knm)`), and the first argument, positionally, is $\mathbf{X}$.

In each demonstration, we show the function signature using X, but we also supply our precomputed kernel for computational efficiency.

## Sparse KPCA with Utility Class

In [ ]:
skpca = SparseKPCA(n_KPCA=2, n_active=n_active, kernel_type=kernel_type)

Calling `skpca.fit(X)` computes the eigendecomposition and internally stores it for projection.

In [ ]:
skpca.fit(X_train, Kmm=Kmm, Knm=Knm_train)

Calling `skpca.transform(X)` computes the projection of $\mathbf{X}$.

In [ ]:
T_skpca = skpca.transform(X_test, Knm=Knm_test)

plot_projection(Y_test, T_skpca, title=f"Sparse KPCA on {n_active} Environments", **cmaps)

`skpca.statistics(X)` returns available statistics. Let's compare to KPCA!

In [ ]:
ref_kpca = KPCA(n_KPCA=n_PCA, kernel_type=kernel_type)
ref_kpca.fit(X_train, K=K_train)

table_from_dict([ref_kpca.statistics(X_train, K=K_train), 
                 skpca.statistics(X_train, Knm=Knm_train),
                 ref_kpca.statistics(X_test, K=K_test), 
                 skpca.statistics(X_test, Knm=Knm_test)], 
                 headers = ["KPCA (Train)", "Sparse KPCA(Train)",
                            "KPCA (Testing)", "Sparse KPCA(Testing)",
                           ], 
                 title="Error in Kernel")

## Sparse KRR with Utility Class

In [ ]:
skrr = SparseKRR(regularization=regularization, n_active=n_active)

Calling `skrr.fit(X,Y)` computes the weights $\mathbf{P}_{KY}$ and internally stores them.

In [ ]:
skrr.fit(X_train, Y_train, Kmm=Kmm, Knm=Knm_train)

Calling `skrr.transform(X)` computes and return the predicted $\mathbf{Y}$ values from $\hat{\mathbf{Y}}_{SKRR} = \mathbf{K}\mathbf{P}_{KY}$.

In [ ]:
Y_skrr_train = skrr.transform(X_train, Knm=Knm_train)
Y_skrr_test = skrr.transform(X_test, Knm=Knm_test)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=dbl_fig)
ref_krr = KRR(regularization=regularization)
ref_krr.fit(X_train, Y_train, K=K_train)
Y_krr = ref_krr.transform(X_test, K=K_test)

plot_regression(Y_train[:,0], Y_skrr_train[:,0], title="Sparse KRR on {} Environments".format(n_active), fig=fig, ax=axes[0], **cmaps)
plot_regression(Y_test[:,0], Y_krr[:,0], title="KRR", fig=fig, ax=axes[1], **cmaps)

fig.subplots_adjust(wspace=0.25)
plt.show()

Calling `skrr.statistics(X,Y)` outputs the statistics of the regression of $\mathbf{X}$ and $\mathbf{Y}$.

We even compare the results of Sparse KRR with our earlier computed KRR.

In [ ]:
table_from_dict([skrr.statistics(X_test, Y_test, Knm=Knm_test), 
                 ref_krr.statistics(X_test, Y_test, K=K_test)], 
                 headers = ["Sparse KRR", "KRR"], 
                 title="KRR Methods: Testing Data")

## Sparse KPCovR from the Utility Class

In [ ]:
alpha = 0.5
regularization=1e-12
skp = SparseKPCovR(alpha=alpha, n_PCA=2, 
                   n_active=n_active, regularization=regularization, 
                   kernel_type=kernel_type)

In [ ]:
skp.fit(X_train, Y_train, Knm=Knm_train, Kmm=Kmm)

In [ ]:
t, y, x = skp.transform(X_test, Knm=Knm_test)

In [ ]:
fig, axes = plt.subplots(1,2, figsize=dbl_fig)

plot_projection(Y_test, t, title = "Sparse KPCovR", **cmaps, fig=fig, ax=axes[0])
plot_regression(Y_test, y, title = "Sparse KPCovR", **cmaps, fig=fig, ax=axes[1])

In [ ]:
ref_kpcovr = KPCovR(alpha=alpha, regularization=regularization, kernel_type=kernel_type)
ref_kpcovr.fit(X_train, Y_train, K=K_train)

table_from_dict([skp.statistics(X_test, Y_test, Knm=Knm_test),
                ref_kpcovr.statistics(X_test, Y_test, K=K_test)],
                headers = ["Sparse KPCovR", "Full KPCovR"])